In [1]:
from darknet import *
from ctypes import *
import cv2

In [2]:
def detect_objects(path):

    # load pre-trained YOLOv3 model & set up Darknet framework
    net = load_net(b'../cfg/yolov3.cfg', b'../weights/yolov3.weights', 0)
    meta = load_meta(b'../cfg/coco.data')

    # feed preprocessed image through YOLOv3 with darknet.detect
    detections = detect(net, meta, path.encode('utf-8'), thresh=.5)

    # return detected objects as a list of tuples, 
    # containing object class and its bounding box coordinates
    return detections

In [3]:

def draw_boxes(path, detections):

    def convert(x, y, w, h):
        x, y, w, h = int(x), int(y), int(w), int(h)
        x1 = x - (w // 2)  # top left x
        y1 = y + (h // 2)  # top left y
        x2 = x + (w // 2)  # bottom right x
        y2 = y - (h // 2)  # bottom right y
        return x1, y1, x2, y2

    # load image from input file path with OpenCV imread
    image = cv2.imread(path)

    # loop over the list of detected objects
    for detection in detections:
        object, confidence, (x, y, w, h) = detection
        x1, y1, x2, y2 = convert(x, y, w, h)

        # draw a rectangle around each object with cv2.rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # put a caption at the corner of each box
        cv2.putText(image, f'{object}: {confidence:.2%}', 
                    (x1, y1 - 5), cv2.LINE_AA, 0.5, (0, 0, 255), 1)

    # display image with bounding boxes with OpenCV imshow
    cv2.imshow('object_detection', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # save image to data directory
    cv2.imwrite('../data/object_detection.jpg', image)

In [4]:
if __name__ == "__main__":
    path = '../data/giraffe.jpg'
    r = detect_objects(path)
    draw_boxes(path, r)

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 